In [ ]:
%pip install --quiet -U langgraph langgraph-checkpoint-postgres psycopg2-binary



In [ ]:
import os
from dotenv import load_dotenv, dotenv_values

# Load environment variables from .env.example
load_dotenv("../.env.example")

def debug_api_key(key_name):
    print(f"\nDebugging {key_name}:")
    
    # Check environment variable
    env_value = os.getenv(key_name)
    print(f"1. Value from os.getenv('{key_name}'): {env_value}")
    
    # Check .env.example file directly
    config = dotenv_values("../.env.example")
    dotenv_value = config.get(key_name)
    print(f"2. Value from .env.example: {dotenv_value}")
    
    # Read .env.example file manually
    try:
        with open("../.env.example", 'r') as f:
            content = f.read()
            print(f"3. Content of .env.example:")
            print(content)
    except FileNotFoundError:
        print("3. Error: .env.example file not found")
    
    # Try to parse the value manually
    if dotenv_value:
        cleaned_value = dotenv_value.strip().strip("'").strip('"')
        print(f"4. Cleaned value: {cleaned_value}")
        
        # Set the environment variable
        os.environ[key_name] = cleaned_value
        print(f"5. Environment variable set. New value: {os.getenv(key_name)}")
    else:
        print("4. Unable to parse value from .env.example")

# Debug both API keys
debug_api_key('GOOGLE_API_KEY')
debug_api_key('LANGCHAIN_API_KEY')

print("\nFinal environment variable values:")
print(f"GOOGLE_API_KEY: {os.getenv('GOOGLE_API_KEY')}")
print(f"LANGCHAIN_API_KEY: {os.getenv('LANGCHAIN_API_KEY')}")

In [112]:
import os
from dotenv import load_dotenv
import psycopg2
from psycopg2.pool import SimpleConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langgraph.graph import END, StateGraph, START
from langgraph.graph import MessagesState
from langgraph.graph.state import CompiledStateGraph

In [113]:
# Database configuration
DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', '127.0.0.1'),
    'port': int(os.getenv('POSTGRES_PORT', 5432)),
    'dbname': os.getenv('POSTGRES_DB', 'postgres'),
    'user': os.getenv('POSTGRES_USER', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD', '123456'),
    'application_name': 'langgraph_app',
    'keepalives': 1,
    'keepalives_idle': 30,
    'keepalives_interval': 10,
    'keepalives_count': 5
}

In [104]:
# Initialize the model
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# State class definition
class State(MessagesState):
    summary: str

# Model calling function
def call_model(state: State) -> State:
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    
    response = model.invoke(messages)
    return {"messages": response}

# Summarization function
def summarize_conversation(state: State) -> State:
    summary = state.get("summary", "")
    
    if summary:
        summary_message = (
            f"This is summary of the conversation to date and answer it accordingly: {summary}\n\n"
            "Extend the summary by taking into account the new messages above and answer it accordingly:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"
    
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Continue function
def should_continue(state: State) -> str:
    messages = state["messages"]
    return "summarize_conversation" if len(messages) > 6 else END

# Create and compile the graph
def create_workflow():
    try:
        workflow = StateGraph(State)
        workflow.add_node("conversation", call_model)
        workflow.add_node(summarize_conversation)
        workflow.add_edge(START, "conversation")
        workflow.add_conditional_edges("conversation", should_continue)
        workflow.add_edge("summarize_conversation", END)
        
        return workflow.compile(checkpointer=memory)
    except Exception as e:
        print(f"Error creating workflow: {str(e)}")
        raise

In [ ]:
# Initialize the graph
graph = create_workflow()

# Configuration for graph invocation
def get_graph_config():
    return {
        "connection": db_manager.get_connection(),
        "configurable": {"thread_id": "1"}
    }

# Example usage
def run_conversation():
    try:
        config = get_graph_config()
        
        # Initial message
        input_message = HumanMessage(content="hi! I'm Lance")
        output = graph.invoke({"messages": [input_message]}, config)
        for m in output['messages'][-1:]:
            m.pretty_print()

        # Follow-up messages
        messages = [
            "what's my name?",
            "i like the 49ers!"
        ]
        
        for message in messages:
            input_message = HumanMessage(content=message)
            output = graph.invoke({"messages": [input_message]}, config)
            for m in output['messages'][-1:]:
                m.pretty_print()
                
    except Exception as e:
        print(f"Error in conversation: {str(e)}")
    finally:
        # Return the connection to the pool
        if config and "connection" in config:
            db_manager.return_connection(config["connection"])

if __name__ == "__main__":
    run_conversation()